# Preliminary analysis: What is in the dataset?

## Importing all the dependencies

In [ ]:
import pandas as pd
%matplotlib inline

## Reading the data

In [ ]:
data = pd.read_csv("data/Evaluaciones_Agropecuarias_Municipales_EVA.csv")

In [ ]:
data.head()

Las variables importantes son: Municipio, Cod Municipio, Grupo de Cultivo, Subgrupo de cultivo, Cultivo, Desagregación regional y/o sistema productivo, año, periodo, rendimiento, ciclo de cultivo

#### Rendimiento se da en toneladas producidas por hectarea cosechada, es decir, entre más rendimiento significa que se sacan más toneladas en un area más pequeña. No tiene nada que ver con el precio, sería bueno conocer el precio del producto

## Changing the column names

In [ ]:
data.columns = ['COD_DEP', 'DEPARTAMENTO', 'COD_MUN','MUNICIPIO','GRUP_CULTIVO','SUBGRUP_CULTIVO','CULTIVO','DESAGREGACION_SIST_PRODUCTIVO','YEAR','PERIODO','AREA_SEMBR','AREA_COSECH','PRODUCCION','RENDIMIENTO','ESTADO_PRODUCCION','NOMBRE_CIENTI','CICLO_CULTIVO']

In [ ]:
data.head()

## Describing the data

In [ ]:
DATA = data 
data.describe()

#### 

In [ ]:
data.info()

## Rendimiento de cultivos por departamento:

In [ ]:
rendimiento_dep = data.groupby('DEPARTAMENTO').RENDIMIENTO.agg(['count','min','max','mean'])
rendimiento_dep.sort_values('mean',ascending=False).head(10)

## Rendimiento de cultivo

In [ ]:
rendimiento_cult = data.groupby('CULTIVO').RENDIMIENTO.agg(['count','min','max','mean'])
rendimiento_cult.sort_values('mean',ascending=False).head(10)

## How many crops types are there?

In [ ]:
rendimiento_cult.count()

## Relevant questions

1. Cuales son los cultivos más propicios a dar un buen rendimiento?
2. Cuales son los departamentos y municipios donde hay mayor rendimiento?
3. Con esto se podrían definir X cultivos que pueden ser aceptados de acuerdo con su rendimiento y el municipio donde se presenta el proyecto

## Example: ¿cuales son los 10 cultivos más propicios para cultivar en Marinilla-Antioquia?

In [ ]:
rendimiento_marinilla = data[data.MUNICIPIO == "MARINILLA"].groupby('CULTIVO').RENDIMIENTO.agg(['count','min','max','mean'])
rendimiento_marinilla.sort_values('mean',ascending=False).head(10)

# Relevant question 1: What are the crops that give the highest yield?

## For Colombia

In [ ]:
stat = data.groupby(data['CULTIVO']).mean()
highest_crops = stat.nlargest(10, 'RENDIMIENTO').reset_index()
highest_crops

In [ ]:
import plotly.express as px
fig = px.bar(highest_crops, x='CULTIVO', y='RENDIMIENTO',color='RENDIMIENTO')
fig.show()

## Yields by department

In [ ]:
rendimiento_dep = data.groupby('DEPARTAMENTO').mean()
print("there are",len(rendimiento_dep),"departments")
highest_department = rendimiento_dep.nlargest(32, 'RENDIMIENTO').reset_index()
highest_department=highest_department.drop(['COD_MUN', 'YEAR'], axis=1)
highest_department.to_csv (r"data/EVA_cultivos_departamento.csv", index = False, header=True)
highest_department

In [ ]:
fig = px.bar(highest_department, x='DEPARTAMENTO', y='RENDIMIENTO',color='RENDIMIENTO')
fig.show()

# Relevant question 2: What are the municipios that have the highest yield

## In general

In [ ]:
rendimiento_mun = data.groupby('MUNICIPIO').RENDIMIENTO.mean()
rendimiento_mun=rendimiento_mun.sort_values(ascending=False).head(10).to_frame().reset_index()
rendimiento_mun

In [ ]:
fig = px.bar(rendimiento_mun, x='MUNICIPIO', y='RENDIMIENTO',color='RENDIMIENTO')
fig.show()

# Relevant question 3: How is the data for a sample?

With this we can just give the algorithm a municipal or department and it will give us:
1. Top 5 Crops
2. Is the yield increasing or decreasing over time
3. Is the sample in the top 5 of country? if its a municipal its in the top 5 of the departament?

In [ ]:
sample = "MARINILLA"
#sample = "Antioquia"
#sample = "Valle del cauca"

## What are the top 5 crops?

In [ ]:
rendimiento_marinilla = data[data.MUNICIPIO == sample].groupby('CULTIVO').RENDIMIENTO.mean()
rendimiento_marinilla=rendimiento_marinilla.sort_values(ascending=False).head(5).to_frame().reset_index()
rendimiento_marinilla
#rendimiento_marinilla.sort_values('mean',ascending=False).head(10)

In [ ]:
fig = px.bar(rendimiento_marinilla, x='CULTIVO', y='RENDIMIENTO',color='RENDIMIENTO')
fig.show()

## How is the yield over time? is it increasing? decreasing?

In [ ]:
rendimiento_marinilla = data[data.MUNICIPIO == sample]
rendimiento_marinilla= rendimiento_marinilla[rendimiento_marinilla.CULTIVO=='REPOLLO']
#fig = px.bar(rendimiento_marinilla, x='YEAR', y='RENDIMIENTO',color='RENDIMIENTO')
#fig.show()

## Is the sample in the top 5 of country? if it's a municipality, is it in the top 5 of the departament?

# change name of departments

In [160]:
#Coger los nombres de departamentos del Json
import json
with open('data/departamentos.json') as f:
    counties = json.load(f)

Departamentos = []
geocol = counties.copy()
#Sacar una lista de los departamentos llamada "Departamentos"
for man in geocol['features']:
    Departamentos.append(man['properties']['NOMBRE_DPT'])
    man['id']=man['properties']['NOMBRE_DPT']

# Abrir Dataframe

df = pd.read_csv("data/Evaluaciones_Agropecuarias_Municipales_EVA.csv")

# Cambiar nombre de columnas para ser más legibles
df.columns = ['COD_DEP', 'DEPARTAMENTO', 'COD_MUN','MUNICIPIO','GRUP_CULTIVO','SUBGRUP_CULTIVO','CULTIVO','DESAGREGACION_SIST_PRODUCTIVO','YEAR','PERIODO','AREA_SEMBR','AREA_COSECH','PRODUCCION','RENDIMIENTO','ESTADO_PRODUCCION','NOMBRE_CIENTI','CICLO_CULTIVO']

# Por cada departamento del Json
for depto in Departamentos:
    # si está ya en el data set, no hacer nada, si no está hacer lo siguiente
    if df['DEPARTAMENTO'].str.contains(depto).any() == False:
        # Revisar cantidad de palabras
        words = depto.split()
        # Si hay una sola palabra, coger el 50 % de esa palabra y buscarlo
        if len(words) == 1:
            df.loc[df.DEPARTAMENTO.str.find(depto[:int(len(depto)*0.6)])>=0, 'DEPARTAMENTO'] = depto
        # Si hay entre 2 o 3 palabras por lo general se encuentra casi la misma palabra (60%)
        elif len(words) > 1 and len(words) <= 3:
            df.loc[df.DEPARTAMENTO.str.find(depto[:int(len(depto)*0.8)])>=0, 'DEPARTAMENTO'] = depto
        # si hay mas de 4 palabras, el unico registro es san andres
        elif len(words) > 4:
            df.loc[df.DEPARTAMENTO.str.find("SAN ANDRES")>=0, 'DEPARTAMENTO'] = depto

# Sacar lista de departamentos por codigo, area sembrada, area cosechada, produccion y rendimiento
# Sacar el promedio de cada dato numerico por departamento
rendimiento_dep = df.groupby('DEPARTAMENTO').mean()
# Cuantos departamentos hay?
print("there are",len(rendimiento_dep),"departments")
# resetear el indice
highest_department = rendimiento_dep.reset_index()
# Quitar columnas que no son necesarias
highest_department=highest_department.drop(['COD_MUN', 'YEAR'], axis=1)
# Pasar a csv
highest_department.to_csv (r"data/EVA_cultivos_departamento.csv", index = False, header=True)
df.to_csv (r"data/EVA_cultivos_modificado.csv", index = False, header=True)

there are 32 departments
